In [3]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Lambda, LSTM, TimeDistributed, Reshape, Bidirectional
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
from keras import backend as K
import matplotlib.pyplot as plt
from oldslidingWindow import read_data, segment_data_by_day, sliding_window
from PROCESSING import pre_processed_data, model_processing_code, model_post_processing
from oldCustom_layers import CustomPenaltyLayer
from sklearn.ensemble import RandomForestClassifier

RAW_DATA = '../Raw Data/Aruba_17/data'
PRE_PROCESSED = '../Processed Data/Aruba_17/pre_processed_data.csv'
FILE_PATH = '../Processed Data/Aruba_17/processed_data.csv'
PREDICTIONS = '../Predictions/Aruba_17_prediction_419.txt'
COMPLETE_PREDICTION = '../Predictions/Aruba_17_completed_prediction_419.txt'
pre_processed_data(RAW_DATA, PRE_PROCESSED)
mappings = model_processing_code(PRE_PROCESSED, FILE_PATH)

data_df = read_data(FILE_PATH)
daily_segments = segment_data_by_day(data_df)
# Use the first 10 days
daily_segments = daily_segments[:32]
# Find the average length of the segments
print(len(daily_segments))
avg_length = 0
for segment in daily_segments:
    avg_length += len(segment)
avg_length /= len(daily_segments)
avg_length = int(avg_length)
print(avg_length)
window_size = avg_length
overlap_ratio = 0.2
windows = sliding_window(daily_segments, window_size=window_size, overlap_ratio=overlap_ratio)
print(len(windows))
# Prepare the data
window_labels = []
windows = np.asarray([window.to_numpy() for window in windows])
for window in windows:
    activity_label = window[0][4]
    activity_status_label = window[0][5]
    label = (activity_label, activity_status_label)
    window_labels.append(label)
# Split the data into training and testing sets
batch_size = 16  
validation_split = 0.3
timesteps = window_size
input_dim = windows[0].shape[1]

Device ID Mapping: {'D001': 0, 'D002': 1, 'D004': 2, 'M001': 3, 'M002': 4, 'M003': 5, 'M004': 6, 'M005': 7, 'M006': 8, 'M007': 9, 'M008': 10, 'M009': 11, 'M010': 12, 'M011': 13, 'M012': 14, 'M013': 15, 'M014': 16, 'M015': 17, 'M016': 18, 'M017': 19, 'M018': 20, 'M019': 21, 'M020': 22, 'M021': 23, 'M022': 24, 'M023': 25, 'M024': 26, 'M025': 27, 'M026': 28, 'M027': 29, 'M028': 30, 'M029': 31, 'M030': 32, 'M031': 33, 'T001': 34, 'T002': 35, 'T003': 36, 'T004': 37, 'T005': 38}
Status Mapping: {'16': 0, '16.5': 1, '17': 2, '17.5': 3, '18': 4, '18.5': 5, '19': 6, '19.5': 7, '20': 8, '20.5': 9, '21': 10, '21.5': 11, '22': 12, '22.5': 13, '23': 14, '23.5': 15, '24': 16, '24.5': 17, '25': 18, '25.5': 19, '26': 20, '26.5': 21, '27': 22, '27.5': 23, '28': 24, '28.5': 25, '29': 26, '29.5': 27, '30': 28, '30.5': 29, '31': 30, '31.5': 31, '32': 32, '32.5': 33, '33': 34, '33.5': 35, '34': 36, '34.5': 37, '35': 38, '35.5': 39, '36': 40, '36.5': 41, '37': 42, '37.5': 43, '38': 44, '38.5': 45, '39': 46,

In [4]:
# Create a new train-test split using the windows
window_train, window_val = train_test_split(windows, test_size=validation_split, shuffle=False)

# Prepare the input data for the model by concatenating the windows along the time axis
X_train = np.concatenate(window_train, axis=0)
X_val = np.concatenate(window_val, axis=0)

# Normalize the data using minMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train = X_train.reshape((-1, timesteps, input_dim))
X_val = X_val.reshape((-1, timesteps, input_dim))

latent_dim = 2
encoding_dim = 32

# ==================== ENCODER ====================
inputs = Input(shape=(timesteps, input_dim), name='encoder_input')
# print('encoder input shape: ', inputs.shape)
x = Bidirectional(LSTM(encoding_dim * 2, return_sequences=True))(inputs)
# print('first encoder bidirectional lstm shape: ', x.shape)
x = Bidirectional(LSTM(encoding_dim, return_sequences=False))(x)
# print('second encoder bidirectional lstm shape: ', x.shape)
z_mean = Dense(latent_dim, name='z_mean')(x)
# print('encoder z_mean shape: ', z_mean.shape)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
# print('encoder z_log_var shape: ', z_log_var.shape)
# z_mean is the mean of the latent space
# z_log_var is the variance of the latent space

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
# print('encoder z shape: ', z.shape)
# ================= LATENT SPACE ==================
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
# print('latent space input shape: ', latent_inputs.shape)
# ==================== DECODER ====================
x = Dense(timesteps * encoding_dim, activation='relu')(latent_inputs)
# print('decoder dense shape: ', x.shape)
x = Reshape((timesteps, encoding_dim))(x)
# print('decoder reshape shape: ', x.shape)
x = Bidirectional(LSTM(encoding_dim, return_sequences=True, input_shape=(timesteps, encoding_dim)))(x)
# print('decoder bidirectional lstm shape: ', x.shape)
x = TimeDistributed(Dense(input_dim))(x)
# print('decoder time distributed dense shape: ', x.shape)
# LSTM layer in the decoder is used to reconstruct the original sequence
# print the summary

# the VAE model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# print('encoder model shape: ', encoder.output_shape)
decoder = Model(latent_inputs, x, name='decoder')
# print('decoder model shape: ', decoder.output_shape)
outputs = decoder(encoder(inputs)[2])
# print('outputs shape: ', outputs.shape)
vae = Model(inputs, outputs, name='vae')
# print('vae model shape: ', vae.output_shape)
# print the summaries
# encoder.summary()
# decoder.summary()
# vae.summary()


# VAE loss function with custom_penalty
reconstruction_loss = K.mean(K.square(inputs - outputs))
reconstruction_loss *= timesteps * input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

# Add the custom penalty to the loss function
penalty_weight = 10.0  # Adjust the weight of the penalty term as needed
penalty_layer = CustomPenaltyLayer(scaler, input_dim, timesteps)
penalty = penalty_layer(outputs)
penalty *= penalty_weight

vae_loss = K.mean(reconstruction_loss + kl_loss + penalty)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
# vae.summary()

In [5]:
num_epochs = 10
history = vae.fit(X_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, None))

# Use the encoder to generate embeddings for each sequence
encoder_model = Model(inputs, z_mean)
X_embedded = encoder_model.predict(X_train, batch_size=batch_size)

Epoch 1/10
1/1 [==============================] - 33s 33s/step - loss: 28974.3652 - val_loss: 118591.0156
Epoch 2/10
1/1 [==============================] - 16s 16s/step - loss: 29927.0234 - val_loss: 118003.0000
Epoch 3/10
1/1 [==============================] - 15s 15s/step - loss: 29580.0527 - val_loss: 117436.2656
Epoch 4/10
1/1 [==============================] - 16s 16s/step - loss: 29210.1895 - val_loss: 116709.2031
Epoch 5/10
1/1 [==============================] - 16s 16s/step - loss: 28765.4316 - val_loss: 115945.3438
Epoch 6/10
1/1 [==============================] - 17s 17s/step - loss: 28310.0801 - val_loss: 114962.5703
Epoch 7/10
1/1 [==============================] - 17s 17s/step - loss: 27678.1621 - val_loss: 113791.1094
Epoch 8/10
1/1 [==============================] - 17s 17s/step - loss: 27053.1914 - val_loss: 112789.8281
Epoch 9/10
1/1 [==============================] - 17s 17s/step - loss: 26333.2383 - val_loss: 111509.9609
Epoch 10/10
1/1 [=============================

In [6]:
# Generate a fake dataset using the VAE model
n_samples = len(windows)

# Sample from the latent space
z_samples = np.random.normal(size=(n_samples, latent_dim))

# Use the decoder to generate the output
predicted_values = decoder.predict(z_samples)
predicted_values = np.reshape(predicted_values, (n_samples, timesteps, input_dim))

# Undo the normalization
predicted_values = np.reshape(predicted_values, (-1, input_dim))
predicted_values = scaler.inverse_transform(predicted_values)

# Round each of the values in the array to the nearest integer
predicted_values = np.rint(predicted_values)

# Create the fake dataset in the original format
fake_dataset = []
for window in predicted_values.reshape((n_samples, timesteps, input_dim)):
    fake_dataset.extend(window)

# Save the fake dataset to a new file 'fake_dataset.txt'
fake_data = pd.DataFrame(fake_dataset, columns=['Date', 'Time', 'Device ID', 'Status', 'Activity', 'Activity Status'])

with open(PREDICTIONS, 'w') as file:
    for _, row in fake_data.iterrows():
        file.write(','.join(map(str, row.values)) + '\n')

model_post_processing(PREDICTIONS, COMPLETE_PREDICTION, mappings)

1/1 [==============================] - 1s 1s/step
         Date          Time  Device ID  Status  Activity  Activity Status
0  20101106.0  1.097294e+10        4.0     3.0       1.0              0.0
1  20101106.0  1.123590e+10        4.0     4.0       1.0              0.0
2  20101106.0  1.118745e+10        4.0     4.0       1.0              0.0
3  20101106.0  1.119466e+10        4.0     4.0       1.0              0.0
4  20101106.0  1.140708e+10        4.0     4.0       1.0              0.0


### Validation

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

y = np.array(window_labels)

def evaluate_model(y_true, y_pred):
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    weighted_precision = precision_score(y_true, y_pred, average='weighted')
    weighted_recall = recall_score(y_true, y_pred, average='weighted')
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
    
    return weighted_f1, weighted_precision, weighted_recall, balanced_accuracy

# Split the embeddings and labels into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_embedded, y, test_size=0.3, shuffle=False)

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_val)

weighted_f1, weighted_precision, weighted_recall, balanced_accuracy = evaluate_model(y_val, y_pred)

print("Weighted F1-score:", weighted_f1)
print("Weighted Precision:", weighted_precision)
print("Weighted Recall:", weighted_recall)
print("Balanced Accuracy:", balanced_accuracy)